<a href="https://colab.research.google.com/github/ArasyH/ArasyH.github.io/blob/main/agentic_patterns_challenge_arasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# !pip install openai-agents
!pip install requests
!pip install groq
!pip install langchain
!pip install langchain-groq

In [3]:
import os
import json
import asyncio
import requests

from typing import List
# from agents import Agent, Runner, function_tool, trace
from langchain_groq import ChatGroq

from google.colab import userdata

SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

headers = {"Authorization": SECTORS_API_KEY}

In [4]:
def retrieve_from_endpoint(url: str) -> dict:

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

In [5]:
from langchain_core.tools import tool


@tool
def get_company_overview(ticker: str, country: str) -> str | None:
    """
    Get company overview from Singapore Exchange (SGX) or Indonesia Exchange (IDX)
    """
    assert country.lower() in ["indonesia", "singapore", "malaysia"], "Country must be either Indonesia, Singapore, or Malaysia"

    if(country.lower() == "indonesia"):
        url = f"https://api.sectors.app/v1/company/report/{ticker}/?sections=overview"
    if(country.lower() == "singapore"):
        url = f"https://api.sectors.app/v1/sgx/company/report/{ticker}/"
    if(country.lower() == "malaysia"):
        url = f"https://api.sectors.app/v1/klse/company/report/{ticker}/"

    try:
        return retrieve_from_endpoint(url)
    except Exception as e:
        print(f"Error occurred: {e}")
        return None


@tool
def get_top_companies_ranked(dimension: str) -> List[str]:
    """
    Return a list of top companies (symbol) based on certain dimension (dividend yield, total dividend, revenue, earnings, market_cap, PB ratio, PE ratio, or PS ratio)
    """

    url = f"https://api.sectors.app/v1/companies/top/?classifications={dimension}&n_stock=3"

    return retrieve_from_endpoint(url)

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.5)

#Agen 1 menyaring perusahaan berdasarkan industries Logistics & Deliveries
screener_tools = [get_company_overview]
screener_prompt = ChatPromptTemplate.from_messages([
    ("system", "Anda adalah asisten yang bertugas mencari perusahaan di industri Logistics & Deliveries. Gunakan tool yang tersedia untuk menemukan ticker perusahaan berdasarkan permintaan pengguna. Hanya berikan output berupa daftar ticker dalam format JSON."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
screener_agent = create_tool_calling_agent(llm, screener_tools, screener_prompt)
screener_executor = AgentExecutor(agent=screener_agent, tools=screener_tools, verbose=True)

# Agen 2 meneliti perusahaan berdasarkan industri
research_tools = [get_company_overview]
researcher_prompt = ChatPromptTemplate.from_messages([
    ("system", "Anda adalah seorang analis riset. Untuk ticker saham yang diberikan, gunakan tool untuk mendapatkan ringkasan perusahaan dan sajikan hasilnya."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

company_research_agent = create_tool_calling_agent(llm, research_tools, researcher_prompt)
researcher_executor = AgentExecutor(agent=company_research_agent, tools=research_tools, verbose=True)

# company_research_agent = Agent(
#     name="company_research_agent",
#     instructions="Research the company by using the right tool",
#     tools=[get_company_overview],
#     output_type=str
# )


In [8]:
async def main():
  """
  CHALLENGE:
  Modify the code here so that you're not hardcoding one stock ticker at a time
  (who have time for that?!) Instead, use a Multi-Agent workflow to have another
  Agent generate a list of companies that you want to research, based on some
  screening condition, then delegate to the second Agent to do the


  CHALLENGE SOLUTION:
    1. Meminta kriteria penyaringan dari pengguna.
    2. Menjalankan 'screener_executor' untuk mendapatkan daftar perusahaan.
    3. Untuk setiap perusahaan dalam daftar, mendelegasikan tugas ke 'researcher_executor'.

  """

  # Langkah 1: Dapatkan kriteria dari pengguna
  screening_condition = input("Masukkan kriteria perusahaan yang ingin Anda cari (contoh: 3 perusahaan dengan market cap tertinggi di Indonesia): ")

  # Langkah 2: Jalankan Agen Penyaring untuk mendapatkan daftar ticker
  print("\n--- MENJALANKAN AGEN PENYARING ---")
  screener_response = await screener_executor.ainvoke({"input": screening_condition})

  try:
      # LLM mungkin mengembalikan string, jadi kita perlu mem-parsingnya ke list
      company_tickers = json.loads(screener_response["output"])
      print(f"\nAgen Penyaring menemukan ticker: {company_tickers}")
  except (json.JSONDecodeError, TypeError):
      print("Gagal mem-parsing daftar ticker dari output agen penyaring.")
      return

  # Langkah 3: Jalankan Agen Peneliti untuk setiap ticker
  print("\n--- MENDELEGASIKAN TUGAS KE AGEN PENELITI ---")
  for ticker in company_tickers:
      print(f"\n--- Meriset {ticker} ---")
      research_response = await researcher_executor.ainvoke({"input": ticker})
      print(f"\n=== Hasil untuk {ticker} ===")
      print(research_response["output"])
      print("=" * 20)


# Menjalankan fungsi utama
await main()

#   company_overview = await Runner.run(
#     company_research_agent, "BBCA"
#   )
#   print(company_overview.final_output)

# await main()

Masukkan kriteria perusahaan yang ingin Anda cari (contoh: 3 perusahaan dengan market cap tertinggi di Indonesia):  Get company overview from Singapore Exchange (SGX) or Indonesia Exchange (IDX)

--- MENJALANKAN AGEN PENYARING ---


> Entering new AgentExecutor chain...


APIError: Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.